<span style="font-size: 35px;">**Waves Lab - Foundation Model Challange!**</span>

Welcome to the **📡Radio Foundation Model** Challenge Notebook! 

This notebook is designed to guide you through the process of adapting our foundational model for various radio tasks. Our model is a Masked Autoencoder (MAE) tailored for radio signal processing built on the vision transformer architecture.

**Objective:**

The primary goal of this challenge is to leverage our model's capabilities to address diverse tasks within the wireless communications domain. Participants are encouraged to fine-tune and adapt the model for applications such as:

- ***Signal Classification:*** Identifying different types of radio signals.
- ***Channel Estimation:*** Predicting the state of communication channels.
- ***Spectrum Sensing:*** Detecting the presence of signals in a frequency band.
- ***Signal Reconstruction:*** Reconstructing signals from incomplete or corrupted data.

## Configuration

⚙️ **Setup Required**

Before running this notebook, please ensure you have configured the necessary parameters in a `.yaml` file similar to `configs/template.yaml` example. This file allows you to define key settings for the fine-tuning experiment, including:

- **Base Model**: Specify the path to the pre-trained model and the desired architecture (`model_path`, `model_arch`).
- **Task-Specific Parameters**: Set the number of classes, input data size, and any additional model-specific configurations (`num_classes`, `input_size`).
- **Dataset Information**: Provide the path to your dataset (`data_path`).
- **Training Hyperparameters**: Adjust batch size, learning rate, number of epochs, and more (`batch_size`, `lr`, `epochs`).
- **Optimizer and Regularization**: Define weight decay, learning rate scheduling, and drop path rate (`weight_decay`, `blr`, `drop_path`).
- **Device and Environment**: Set the device type (`cuda` or `cpu`), random seed, and data loader options (`device`, `seed`, `num_workers`).

🔔 Before proceeding with the setup, you'll need to define some key parameters for your fine-tuning experiment. Please make sure to configure the following variables:

1. **base_model_arch**:  
   Choose the pre-trained model architecture you'd like to fine-tune. The available options are:
   - `seg_vit_small_patch16`
   - `seg_vit_medium_patch16`
   - `seg_vit_large_patch16`

2. **base_model_path**:  
   Provide the path to the selected pre-trained model architecture on your system.

3. **task**:  
   Select the task you want to fine-tune the base model for. The available tasks are:
   - `Segmentation`
   - `Sensing`
   - `Custom` (for your own dataset and custom processing functions)

4. **data_path**:  
   Specify the path to the dataset on your system for the chosen task. This should be the dataset required for your fine-tuning.
   
Once you have configured these parameters, set the `config_path` variable in the next cell with the path of that `.yaml` file to proceed with running the notebook.

**📝 Edit the `.yaml` file now!**

In [1]:
import yaml

config_path = "/home/elsayedmohammed/mae/configs/positioning.yaml"    # TODO: Set
config_path = "/home/elsayedmohammed/mae/configs/channel_estimation.yaml"

with open(config_path, 'r') as yaml_file:
    config = yaml.safe_load(yaml_file)

## Loading the configuration

In [2]:
import os 
import torch
import torch.backends.cudnn as cudnn
import numpy as np
from FineTuningArgs import FineTuningArgs

exp_name = config["experiment_name"]
del config["experiment_name"]

data_path = config["data_path"]
assert os.path.isdir(data_path), print(f"Incorrect data_path! ({data_path})")
print(f"The dataset path provided: {data_path}")

output_dir = os.path.join(config["output_dir"], exp_name)
os.makedirs(output_dir, exist_ok=True)

print(f"The outputs path provided: {output_dir}")

print(f"==== Loading all the configs..")
config = FineTuningArgs(**config)

print(f"==== Setting the device and random seed..")
device = torch.device(config.device)
print(f"Device: {device}")

torch.manual_seed(config.seed)
np.random.seed(config.seed)

cudnn.benchmark = True # DEVELOPERS:check

The dataset path provided: /home/elsayedmohammed/datasets/channel_estimation_dataset
The outputs path provided: /home/elsayedmohammed/outputs/exp1_channel_estimation
==== Loading all the configs..
Finetuning on the (channel_estimation) task..
==== Setting the device and random seed..
Device: cuda


## Dataset

In [3]:
if config.task == 'segmentation':
    from dataset_classes.segmentation_dataset import SegmentationDataset as TaskDataset
elif config.task == 'sensing':
    from dataset_classes.csi_sensing_dataset import CSISensingDataset as TaskDataset
elif config.task == 'signal_identification':
    from dataset_classes.radio_signal_identification_dataset import SignalIdentificatio_Dataset as TaskDataset
elif config.task == 'positioning':
    from dataset_classes.positioning_nr_dataset import PositioningNR as TaskDataset
elif config.task == 'channel_estimation':
    from dataset_classes.ofdm_channel_estimation_dataset import OfdmChannelEstimation as TaskDataset
else:
    # TODO
    assert False, print("Replace this line with import statment \
                         for your dataset class as TasDataset")
    # You can also build your dataset class here in this cell and then change the two following lines accordingly

dataset_train = TaskDataset(data_path, split="train")
dataset_val = TaskDataset(data_path, split="val")


Load the dataset into Train and Val objects.

In [4]:
# For Training dataset
## 1. Create the sampling object (Training)
sampler_train = torch.utils.data.RandomSampler(dataset_train)
## 2. Create the dataloader (Training)
data_loader_train = torch.utils.data.DataLoader(
        dataset_train, sampler=sampler_train,
        batch_size=config.batch_size,
        num_workers=config.num_workers,
        pin_memory=config.pin_mem,
        drop_last=True,
    )

# For Valdiation dataset
## 1. Create the sampling object (Validation)
sampler_val = torch.utils.data.SequentialSampler(dataset_val)
## 2. Create the dataloader (Validation)
data_loader_val = torch.utils.data.DataLoader(
        dataset_val, sampler=sampler_val,
        batch_size=config.batch_size,
        num_workers=config.num_workers,
        pin_memory=config.pin_mem,
        drop_last=False
    )

## Model

In [5]:
if config.task == 'segmentation':
    import models.segmentation as task_model
    assert config.base_arch in list(task_model.__dict__.keys()),\
        print(f"This model architecture ({config.base_arch}) is not available!")
    model = task_model.__dict__[config.base_arch]() 

elif config.task == 'sensing':
    import models.sensing as task_model
    assert config.base_arch in list(task_model.__dict__.keys()),\
        print(f"This model architecture ({config.base_arch}) is not available!")
    model = task_model.__dict__[config.base_arch](global_pool=config.global_pool,
                                                num_classes=config.num_classes,
                                                drop_path_rate=config.drop_path)
    
elif config.task == 'signal_identification':
    import models.signal_identification as task_model
    assert config.base_arch in list(task_model.__dict__.keys()),\
        print(f"This model architecture ({config.base_arch}) is not available!")
    model = task_model.__dict__[config.base_arch](global_pool=config.global_pool,
                                                num_classes=config.num_classes,
                                                drop_path_rate=config.drop_path,
                                                in_chans=1)
elif config.task == 'positioning':
    print("POSITIONING")
    scene = "outdoor" # TODO: (DEVELOPERS)
    tanh = False # TODO: (DEVELOPERS)
    import models.positioning as task_model
    assert config.base_arch in list(task_model.__dict__.keys()),\
        print(f"This model architecture ({config.base_arch}) is not available!")
    model = task_model.__dict__[config.base_arch](global_pool=config.global_pool, num_classes=config.num_classes,
                                            drop_path_rate=config.drop_path, tanh=tanh,
                                            in_chans=4 if scene == 'outdoor' else 5)
elif config.task == 'channel_estimation':
    import models.channel_estimation as task_model
    assert config.base_arch in list(task_model.__dict__.keys()),\
        print(f"This model architecture ({config.base_arch}) is not available!")
    model = task_model.__dict__[config.base_arch]() 

else:
    # TODO
    assert False, print("Replace this line with import statment \
                         for your model class as task_model")
    # You can also build your model class here in this cell and then change the two following lines accordingly
    #  
# Load the model checkpoint
print(f"Loading pre-trained checkpoint from: {config.base_model_path} ...")
msg = model.load_model_checkpoint(checkpoint_path=config.base_model_path)
print(msg) # TODO- (DEVELOPERS): why In_IncompatibleKeys?

# Freeze the encoder weights (the base)
model.freeze_encoder()
model.to(device) 

# Check the model's number of parameters
n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('Number of params (M): %.2f' % (n_parameters / 1.e6))

/home/elsayedmohammed/AoA-Pruning/pruning-venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading pre-trained checkpoint from: /home/elsayedmohammed/vit-models/pretrained_medium_75.pth ...


/home/elsayedmohammed/mae/models/channel_estimation.py:175: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location='cpu')


Removing key decoder_pred.weight from pretrained checkpoint
Removing key decoder_pred.bias from pretrained checkpoint
_IncompatibleKeys(missing_keys=['decoder_pred.weight', 'decoder_pred.bias'], unexpected_keys=['mask_token', 'decoder_blocks.1.norm1.weight', 'decoder_blocks.1.norm1.bias', 'decoder_blocks.1.attn.qkv.weight', 'decoder_blocks.1.attn.qkv.bias', 'decoder_blocks.1.attn.proj.weight', 'decoder_blocks.1.attn.proj.bias', 'decoder_blocks.1.norm2.weight', 'decoder_blocks.1.norm2.bias', 'decoder_blocks.1.mlp.fc1.weight', 'decoder_blocks.1.mlp.fc1.bias', 'decoder_blocks.1.mlp.fc2.weight', 'decoder_blocks.1.mlp.fc2.bias', 'decoder_blocks.2.norm1.weight', 'decoder_blocks.2.norm1.bias', 'decoder_blocks.2.attn.qkv.weight', 'decoder_blocks.2.attn.qkv.bias', 'decoder_blocks.2.attn.proj.weight', 'decoder_blocks.2.attn.proj.bias', 'decoder_blocks.2.norm2.weight', 'decoder_blocks.2.norm2.bias', 'decoder_blocks.2.mlp.fc1.weight', 'decoder_blocks.2.mlp.fc1.bias', 'decoder_blocks.2.mlp.fc2.weig

## Optimizer

In [6]:
# TODO: Feel free to set your own loss function or LR scheduler

import util.lr_decay as lrd
from util.misc import NativeScalerWithGradNormCount as NativeScaler
from timm.loss import LabelSmoothingCrossEntropy

param_groups = lrd.param_groups_lrd(model, config.weight_decay, layer_decay=config.layer_decay)
optimizer = torch.optim.AdamW(param_groups, lr=config.lr)
loss_scaler = NativeScaler()

if config.smoothing > 0.:
    criterion = LabelSmoothingCrossEntropy(smoothing=config.smoothing)
else:
    criterion = torch.nn.CrossEntropyLoss()

if config.task == "channel_estimation":
    from torch.nn import MSELoss
    criterion = MSELoss()

print(f"criterion selected: {str(criterion)}")

criterion selected: MSELoss()


/home/elsayedmohammed/mae/util/misc.py:76: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self._scaler = torch.cuda.amp.GradScaler()


# Finetuning

In [7]:
if config.task == "segmentation":
    from finetuning_engines.segmentation import train_one_epoch, evaluate
elif config.task in ["sensing", "signal_identification"]:
    from finetuning_engines.sensing import train_one_epoch, evaluate
elif config.task == "channel_estimation":
    from finetuning_engines.channel_estimation import train_one_epoch, evaluate
else:
    # TODO
    assert False, print("Replace this line with import statment \
                         for your finetuing engine script with train_one_epoch and evaluate functions")
    # You can also build your functions here in this cell.

In [8]:
import time
import util.misc as misc
import json
import datetime

print(f"Training for {config.epochs} epochs..")
start_time = time.time()
least_val_loss, best_stats_epoch = np.inf, 0

for epoch in range(config.epochs):
    train_stats = train_one_epoch(
        model, criterion, data_loader_train,
        optimizer, device, epoch, loss_scaler,
        config.clip_grad, None,
        args=config
    )
    if config.output_dir and (epoch % config.save_every == 0):
        misc.save_model(args=config, model=model, optimizer=optimizer, loss_scaler=loss_scaler, epoch=epoch)

    val_stats = evaluate(data_loader_val, model, criterion, device)
    if val_stats["avg_loss"] < least_val_loss:
         least_val_loss = val_stats["avg_loss"]
         best_stats_epoch = epoch

    if config.output_dir:
        log_stats = {"epoch": epoch,
                     "train_loss": train_stats["avg_loss"],
                     "val_loss": val_stats["avg_loss"],
                     "val_acc": val_stats["avg_acc"]}
        with open(os.path.join(config.output_dir, "log.txt"), mode="a", encoding="utf-8") as f:
            f.write(json.dumps(log_stats) + "\n")

total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))

Training for 10 epochs..


Epoch 0/10:   0%|          | 0/25 [00:00<?, ?batch/s]/home/elsayedmohammed/AoA-Pruning/pruning-venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([1376256])) that is different to the input size (torch.Size([688128, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch 0/10:   0%|          | 0/25 [00:16<?, ?batch/s]


RuntimeError: The size of tensor a (2) must match the size of tensor b (1376256) at non-singleton dimension 1

# Reporting

The last step is to report the finetuning results in the following `.txt` format to enable evaluation.

Our evaluation takes into account:
 - Number of data samples
 - Number of model parameters
 - The reported loss/accuracy

**Required Format:**
```
{
    task:               'task_name(arbitrary)',
    loss:               validation final loss (if regression) or None otherwise,
    accuracy:           validation final accurcay (if classification) or None otherwise,
    model_n_parameters: number of model parameters,
    validation_length:  number of data samples in your validation dataset
    score:              the score assigned to your task (automatically generated)
}
```

In [ ]:
from util.misc import report_score

report = report_score(config, model, dataset_val, least_val_loss, None)

In [ ]:
# TODO (DEVELOPERS): Just for us, going to delete this cell soon!

validation_length = len(dataset_val)
print(f"validation_length = {validation_length}")

validation_score = min(validation_length // 100 * 10, 100)
print(f"validation_score = {validation_score}")

print(least_val_loss)
print(f"performance (loss) = {least_val_loss}")

perf_score = 100 - least_val_loss*100
print(f"paras_score = {perf_score}")

model_n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"parameteres = {model_n_parameters}")

paras_score = 100 - min(max(model_n_parameters // 200000 * 5, 0), 100) 
print(f"paras_score = {paras_score}")

print(f"===total = {0.5*perf_score + 0.25*paras_score + 0.25*validation_score}")
